# Make the Gregory style plot for C1 scenarios for TCRE and ZEC from CO2 only runs

In [ ]:
import logging

import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
from scipy.stats import linregress

In [ ]:
logger = logging.getLogger('fair')
logger.setLevel(level=logging.CRITICAL)

In [ ]:
pl.style.use('../defaults.mplstyle')

In [ ]:
emis_future = np.load("../data/co2_emissions_c1.npy")  # C1 emissions scenarios starting in 1995

In [ ]:
emis_future.shape

In [ ]:
# First, remove three scenarios which don't reach net zero
emis_future[-1,:] > 0

In [ ]:
emis_future = emis_future[:,emis_future[-1,:] < 0]

In [ ]:
emis_future.shape

In [ ]:
# then, ensure we pull in the full historical
emis = np.ones((351, 94)) * np.nan

In [ ]:
df_hist = pd.read_csv('../data/historical_emissions.csv')
emis_hist = (
    df_hist.loc[df_hist['Variable']=='Emissions|CO2|MAGICC AFOLU', '1750':'1994'].values +
    df_hist.loc[df_hist['Variable']=='Emissions|CO2|MAGICC Fossil and Industrial', '1750':'1994'].values
).squeeze()/1000

In [ ]:
emis[:245, :] = emis_hist[:, None]

In [ ]:
emis[245:, :] = emis_future

In [ ]:
first_negative_year = np.argmin(emis>0, axis=0)

In [ ]:
first_negative_year  # bear in mind these are "timepoints" so are year+.5

In [ ]:
# the appropriate timebound is the one with the same label as first_negative_year, occurring at the whole number year
emis[first_negative_year[0]-1, 0], emis[first_negative_year[0], 0]

In [ ]:
# cumulative net negative CO2 emissions from year of net zero to 2100 (GtCO2)
cumulative_positive_1750_to_nz = np.ones(94) * np.nan
cumulative_negative_nz_to_2100 = np.ones(94) * np.nan

for i, year in enumerate(first_negative_year):
    cumulative_positive_1750_to_nz[i] = np.cumsum(emis[:year, i])[-1] / 1000
    cumulative_negative_nz_to_2100[i] = np.cumsum(emis[year:, i])[-1] / 1000

In [ ]:
# this suggests that the split is correct
np.cumsum(emis[:year, i])[-1]/1000, np.cumsum(emis[year:, i])[0] / 1000

In [ ]:
cumulative_positive_1750_to_nz  # TtCO2 each scenario (x94) 1750 to net zero

In [ ]:
cumulative_negative_nz_to_2100  # unit: TtCO2 in each scenario (x94) from net zero to 2100

## now run fair and calculate warming at different points
- 1750 to peak T (positive phase)
- 1750 to T @ net zero (positive phase)
- peak T to end of century temperature (negative phase)
- T @ net zero to end of century (negative phase)

As with cumulative emissions, the net zero point should be determined using the same label as net zero year (timebound before timepoint where first negative value encountered)

In [ ]:
f = FAIR()
scenarios = np.arange(94)
f.define_scenarios(scenarios)

In [ ]:
fair_params_1_4_1_file = '../data/calibrated_constrained_parameters_1.4.1.csv'
fair_species_configs_1_4_1_file = '../data/species_configs_properties_1.4.1.csv'

In [ ]:
df_configs = pd.read_csv(fair_params_1_4_1_file, index_col=0)
configs = df_configs.index  # this is used as a label for the "config" axis
f.define_configs(configs)

In [ ]:
species = ['CO2', 'CH4', 'N2O']
properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
f.define_species(species, properties)

In [ ]:
f.define_time(1750, 2101, 1)

In [ ]:
f.allocate()

In [ ]:
f.concentration.loc[dict(specie="CH4")] = 729.2
f.concentration.loc[dict(specie="N2O")] = 270.1

In [ ]:
f.emissions.loc[dict(specie="CO2")] = emis[..., None]

In [ ]:
f.fill_species_configs(fair_species_configs_1_4_1_file)

In [ ]:
f.override_defaults(fair_params_1_4_1_file)

In [ ]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.ocean_heat_content_change, 0)

In [ ]:
f.run()

In [ ]:
f.temperature.sel(layer=0).shape

In [ ]:
temp_peakT_to_2100 = (
    f.temperature.sel(layer=0, timebounds=2101) -
    f.temperature.sel(layer=0).max(dim='timebounds')
).values  # convention negative for cooling

In [ ]:
#pl.plot(f.temperature.sel(layer=0, scenario=0));

In [ ]:
temp_1750_to_peakT = (
    f.temperature.sel(layer=0).max(dim='timebounds') -
    f.temperature.sel(layer=0, timebounds=1750)  # 1750 T should be zero
).values

In [ ]:
temp_peakT_to_2100

In [ ]:
temp_1750_to_peakT

In [ ]:
temp_nz_to_2100 = np.ones((94, 841)) * np.nan
for i in range(94):
    temp_nz_to_2100[i, :] = (
        f.temperature.sel(layer=0, scenario=i, timebounds=2101) -
        f.temperature.sel(layer=0, scenario=i, timebounds=1750+first_negative_year[i])
    ).values

temp_1750_to_nz = np.ones((94, 841)) * np.nan
for i in range(94):
    temp_1750_to_nz[i, :] = (
        f.temperature.sel(layer=0, scenario=i, timebounds=1750+first_negative_year[i]) - 
        f.temperature.sel(layer=0, scenario=i, timebounds=1750)
    ).values

In [ ]:
temp_nz_to_2100

In [ ]:
temp_1750_to_nz

In [ ]:
# TCRE_down is K per EgCO2: temperature divide cumulative emissions
# notice that this method doesn't work where ZEC is strongly positive, since 2101 is highest temperature, and values bunch at zero
pl.hist(temp_peakT_to_2100[0, :] / cumulative_negative_nz_to_2100[0], bins=20)
pl.title('post-peak T negative TCRE (°C/TtCO2 removed, 1st scen.)')
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
pl.hist(temp_nz_to_2100[0, :] / cumulative_negative_nz_to_2100[0], bins=20)
pl.title('post-netzero negative TCRE (°C/TtCO2 removed, 1st scen.)')
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
# we see the post net zero definition doesn't suffer from problem of positive ZEC
# in fact, ZEC is aliased into the definition, which is kind of what we want
pl.hist(temp_peakT_to_2100[0, :] / cumulative_negative_nz_to_2100[0], bins=np.arange(-0.75, 1.3, 0.05), alpha=0.4, label='post peakT')
pl.hist(temp_nz_to_2100[0, :] / cumulative_negative_nz_to_2100[0], bins=np.arange(-0.75, 1.3, 0.05), alpha=0.4, label='post net zero')
pl.title('comparison defs. -ve TCRE (°C/TtCO2 removed, 1st scen.)')
pl.legend()
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
pl.hist(temp_1750_to_peakT[0, :] / cumulative_positive_1750_to_nz[0], bins=np.arange(0.20, 0.82, 0.02), alpha=0.4, label='up to peakT')
pl.hist(temp_1750_to_nz[0, :] / cumulative_positive_1750_to_nz[0], bins=np.arange(0.20, 0.82, 0.02), alpha=0.4, label='up to net zero')
pl.title('comparison defs. +ve TCRE (°C/TtCO2 removed, 1st scen.)')
pl.legend()
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
pl.hist(
    (temp_peakT_to_2100[0, :] / cumulative_negative_nz_to_2100[0]) / (temp_1750_to_peakT[0, :] / cumulative_positive_1750_to_nz[0]), 
bins=np.arange(-1.1, 2.7, 0.1), alpha=0.4, label='post peakT')
pl.hist(
    (temp_nz_to_2100[0, :] / cumulative_negative_nz_to_2100[0]) / (temp_1750_to_nz[0, :] / cumulative_positive_1750_to_nz[0]), 
bins=np.arange(-1.1, 2.7, 0.1), alpha=0.4, label='post netzero')
#pl.hist(temp_1750_to_nz[0, :] / cumulative_positive_1750_to_nz[0], bins=np.arange(-0.75, 1.3, 0.05), alpha=0.4, label='post net zero')
pl.title('comparison of negative/positive TCRE ratios (1st scen.)')
pl.legend()
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
# we see where we hit net zero late, we have much larger uncertainty in TCRE since cumulative CO2 divisor is small
pl.hist(temp_peakT_to_2100[30, :] / cumulative_negative_nz_to_2100[30], bins=np.arange(-4, 5.1, 0.1), alpha=0.4, label='post peakT')
pl.hist(temp_nz_to_2100[30, :] / cumulative_negative_nz_to_2100[30], bins=np.arange(-4, 5.1, 0.1), alpha=0.4, label='post net zero')
pl.title('comparison defs. -ve TCRE (°C/TtCO2 removed, 31st scen.)')
pl.legend()
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
cmap = mpl.colormaps['plasma']
# Take colors at regular intervals spanning the colormap.
colors = cmap(np.linspace(0, 1, 841))

In [ ]:
# color by tcre_up_at_netzero
for i in range(841):
    pl.scatter(cumulative_negative_nz_to_2100, temp_peakT_to_2100[:, i], s=1, color=colors[i])
pl.title('All scenarios, each fair config')
pl.xlabel('netzero cum. removal (net zero to 2100, 1000GtCO2)')
pl.ylabel('warming, peak temperature to 2100')

In [ ]:
# each coloured band is a parameter set from fair (dim 841), each vertical line is a C1 scenario (dim 94)
sl_down_nz = np.zeros(841)
ic_down_nz = np.zeros(841)
for i in range(841):
    pl.scatter(cumulative_negative_nz_to_2100, temp_nz_to_2100[:, i], s=1)
    lr = linregress(cumulative_negative_nz_to_2100, temp_nz_to_2100[:, i])
    sl_down_nz[i], ic_down_nz[i] = lr.slope, lr.intercept
pl.title('All scenarios, each fair config')
pl.xlabel('netzero cum. removal (net zero to 2100, 1000GtCO2)')
pl.ylabel('warming, year of net zero to 2100')

In [ ]:
sl_up_nz = np.zeros(841)
ic_up_nz = np.zeros(841)
for i in range(841):
    pl.scatter(cumulative_positive_1750_to_nz, temp_1750_to_nz[:, i])
    lr = linregress(cumulative_positive_1750_to_nz, temp_1750_to_nz[:, i])
    sl_up_nz[i], ic_up_nz[i] = lr.slope, lr.intercept
pl.title('All scenarios, each fair config')
pl.xlabel('cum. emissions (1750 to net zero, 1000GtCO2)')
pl.ylabel('warming, 1750 to year of net zero')

In [ ]:
pl.hist(sl_up_nz, bins=np.arange(0.1, 0.82, 0.02), alpha=0.4, label='TCREup')
pl.hist(sl_down_nz, bins=np.arange(0.2, 0.72, 0.02), alpha=0.4, label='TCRE_down')
pl.title('Regression based estimates of TCREup and TCREdown')
print(np.percentile(sl_up_nz, (5, 50, 95)))
print(np.percentile(sl_down_nz, (5, 50, 95)))
pl.legend()

In [ ]:
pl.hist(ic_up_nz, bins=np.arange(-0.3, 1.02, 0.02), alpha=0.4, label='TCREup')
pl.hist(ic_down_nz, bins=np.arange(-0.3, 1.02, 0.02), alpha=0.4, label='TCRE_down')
pl.title('Regression-based ZEC estimates from netzero')
print(np.percentile(ic_up_nz, (5, 50, 95)))
print(np.percentile(ic_down_nz, (5, 50, 95)))
pl.legend()

In [ ]:
2/2.7

In [ ]:
0.6/2.7

In [ ]:
# suggestion is to calculate the positive TCRE either from 1pct runs or just by taking temperature at net zero 
# divide cumulative emissions at net zero
# it becomes difficult because ZEC itself is defined as a point estimate in 1pctCO2 runs
# I think the most approprite value to use for TCREup is the total warming at net zero divide cumulative emissions at net zero.
# then rely on the fact that ZEC in fair is ~zero, and take ratio of TCREdown and TCREup.

In [ ]:
for iconf in range(841):
    pl.plot(np.cumsum(emis[:, 0]), f.temperature.sel(timebounds=np.arange(1751, 2102), layer=0, scenario=0, config=configs[iconf]));

In [ ]:
cumulative_positive_1750_to_nz

In [ ]:
# one TCRE estimate for every scenario/config; but scenarios look similar
tcre_up_at_net_zero = temp_1750_to_nz/cumulative_positive_1750_to_nz[:, None]

In [ ]:
tcre_up_at_net_zero.shape

In [ ]:
for i in range(94):
    pl.hist(tcre_up_at_net_zero[:, i], alpha=0.3)

In [ ]:
pl.hist(np.mean(tcre_up_at_net_zero, axis=0))

In [ ]:
pl.hist(np.mean(tcre_up_at_net_zero * 3.664, axis=0), bins=np.arange(0.3, 3.05, 0.05), alpha=0.4, label='TCREup')
pl.hist(sl_down_nz * 3.664, bins=np.arange(0.3, 3.05, 0.05), alpha=0.4, label='TCRE_down')
pl.title('Regression based estimates of TCREup and TCREdown')
print(np.percentile(np.mean(tcre_up_at_net_zero * 3.664, axis=0), (5, 50, 95)))
print(np.percentile(sl_down_nz * 3.664, (5, 50, 95)))
pl.legend()

In [ ]:
# ratio
pl.hist(sl_down_nz / np.mean(tcre_up_at_net_zero, axis=0))
np.percentile(sl_down_nz / np.mean(tcre_up_at_net_zero, axis=0), (5, 50, 95))